In [45]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import tensorflow_hub as hub
import os

In [42]:
#決定主要變數
img_size = 64
batch_size = 128
init_filter = 64
epoch = 50
cardinality = 0
features = '-original'

#DataAugmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20
    )

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#載入資料集
train_set = train_datagen.flow_from_directory(
    'train',
    target_size=(img_size, img_size),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False   
)


valid_set = test_datagen.flow_from_directory(
    'valid',
    target_size=(img_size, img_size),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)

#不會加入訓練，僅作為模型表現之參考
test_set = test_datagen.flow_from_directory(
    'test',
    target_size=(img_size, img_size),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


In [32]:
#觀察許多結構有類似表現，因此特別寫一個函數出來，避免後續程式碼過於冗長
def transform_layer(x, filters, kernel_size):
    out = layers.Conv2D(filters, kernel_size, padding='same')(x)
    out = layers.BatchNormalization()(out)
    out = layers.ReLU()(out)
    return out

#題目中左邊（cardinality=0，同時是後續實作Resnet50中的第二種bottleneck）與中間之residual block（cardinality!=0）
def Residual_Block_1_2(inputs, filters, cardinality):
    x = layers.Conv2D(filters,1)(inputs)
    fx = transform_layer(inputs, filters/4, 1)   
    if cardinality != 0:
        layers_split = list()
        for i in range(cardinality):
            splits = transform_layer(fx, filters/4, 3)
            layers_split.append(splits)
        layers_split = tf.keras.layers.concatenate(layers_split)
        fx = transform_layer(layers_split, filters, 3)
    else:
        fx = transform_layer(fx,filters, 3)
        
    fx = layers.Conv2D(filters, 1, padding='same')(fx)
    fx = layers.BatchNormalization()(fx)
    outputs = layers.Add()([x,fx])
    outputs = layers.ReLU()(outputs)
    return outputs
    
#題目中右邊之residual block
def Residual_Block_3(inputs, filters):
    x = layers.Conv2D(filters,1)(inputs)    
    fx = transform_layer(inputs, filters/4,1)
    fx1 = transform_layer(fx, filters/4,3)
    fx2 = transform_layer(fx, filters/4,3)
    fx3 = transform_layer(fx, filters/4,1)
    
    fx1 = tf.keras.layers.GlobalAveragePooling2D()(fx1)
    fx4 = tf.keras.layers.Multiply()([fx1,fx2])
    fx4 = transform_layer(fx4, filters,1)
    
    fx5 = tf.keras.layers.concatenate([fx3,fx4])
    fx5 = layers.Conv2D(filters, 3, padding='same')(fx5)
    fx5 = layers.BatchNormalization()(fx5)
    
    outputs = layers.Add()([x,fx4,fx5])
    outputs = layers.ReLU()(outputs)
    
    return outputs   

#後續實作Resnet50中的第一種bottleneck，且加入strides為變數
def BTNK_1(inputs, filters, strides):
    x = layers.Conv2D(int(4*filters/strides), 1, strides=strides)(inputs)
    fx = transform_layer(inputs, int(filters/strides), 1)   
    fx = transform_layer(fx, int(filters/strides), 3)       
    fx = layers.Conv2D(int(4*filters/strides), 1, padding='same',strides=strides)(fx)
    fx = layers.BatchNormalization()(fx)
    outputs = layers.Add()([x,fx])
    outputs = layers.ReLU()(outputs)
    return outputs

In [43]:
print(img_size,batch_size,init_filter,epoch,cardinality)
#Resnet50_stage_input
inputs = keras.Input(shape=(img_size, img_size, 3))

#Resnet50_stage_0
x = layers.Conv2D(init_filter,2,strides=2)(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPool2D(pool_size=3, strides=2,padding='same')(x)

#Resnet50_stage_1
x = BTNK_1(x, init_filter, 1)
x = Residual_Block_1_2(x, 4*init_filter, cardinality)
x = Residual_Block_1_2(x, 4*init_filter, cardinality)

#Resnet50_stage_2
x = BTNK_1(x, init_filter, 2)
x = Residual_Block_1_2(x, 8*init_filter, cardinality)
x = Residual_Block_1_2(x, 8*init_filter, cardinality)
x = Residual_Block_1_2(x, 8*init_filter, cardinality)

#Resnet50_stage_3
x = BTNK_1(x, init_filter, 2)
x = Residual_Block_1_2(x, 16*init_filter, cardinality)
x = Residual_Block_1_2(x, 16*init_filter, cardinality)
x = Residual_Block_1_2(x, 16*init_filter, cardinality)
x = Residual_Block_1_2(x, 16*init_filter, cardinality)
x = Residual_Block_1_2(x, 16*init_filter, cardinality)

#Resnet50_stage_4
x = BTNK_1(x, init_filter, 2)
x = Residual_Block_1_2(x, 32*init_filter, cardinality)
x = Residual_Block_1_2(x, 32*init_filter, cardinality)

#Resnet50_stage_output
x = layers.AveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
outputs = layers.Dense(11, activation='softmax')(x)
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

128 64 64 50 0
Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_857 (Conv2D)            (None, 64, 64, 64)   832         ['input_14[0][0]']               
                                                                                                  
 batch_normalization_664 (Batch  (None, 64, 64, 64)  256         ['conv2d_857[0][0]']             
 Normalization)                                                                                   
                                                                            

                                                                                                  
 conv2d_866 (Conv2D)            (None, 32, 32, 256)  65792       ['re_lu_670[0][0]']              
                                                                                                  
 batch_normalization_673 (Batch  (None, 32, 32, 256)  1024       ['conv2d_869[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_195 (Add)                  (None, 32, 32, 256)  0           ['conv2d_866[0][0]',             
                                                                  'batch_normalization_673[0][0]']
                                                                                                  
 re_lu_673 (ReLU)               (None, 32, 32, 256)  0           ['add_195[0][0]']                
          

                                                                                                  
 add_198 (Add)                  (None, 16, 16, 512)  0           ['conv2d_878[0][0]',             
                                                                  'batch_normalization_682[0][0]']
                                                                                                  
 re_lu_682 (ReLU)               (None, 16, 16, 512)  0           ['add_198[0][0]']                
                                                                                                  
 conv2d_883 (Conv2D)            (None, 16, 16, 128)  65664       ['re_lu_682[0][0]']              
                                                                                                  
 batch_normalization_683 (Batch  (None, 16, 16, 128)  512        ['conv2d_883[0][0]']             
 Normalization)                                                                                   
          

                                                                                                  
 conv2d_895 (Conv2D)            (None, 8, 8, 256)    262400      ['re_lu_691[0][0]']              
                                                                                                  
 batch_normalization_692 (Batch  (None, 8, 8, 256)   1024        ['conv2d_895[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_692 (ReLU)               (None, 8, 8, 256)    0           ['batch_normalization_692[0][0]']
                                                                                                  
 conv2d_896 (Conv2D)            (None, 8, 8, 1024)   2360320     ['re_lu_692[0][0]']              
                                                                                                  
 batch_nor

                                                                                                  
 re_lu_701 (ReLU)               (None, 8, 8, 256)    0           ['batch_normalization_701[0][0]']
                                                                                                  
 conv2d_908 (Conv2D)            (None, 8, 8, 1024)   2360320     ['re_lu_701[0][0]']              
                                                                                                  
 batch_normalization_702 (Batch  (None, 8, 8, 1024)  4096        ['conv2d_908[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_702 (ReLU)               (None, 8, 8, 1024)   0           ['batch_normalization_702[0][0]']
                                                                                                  
 conv2d_90

 batch_normalization_711 (Batch  (None, 4, 4, 2048)  8192        ['conv2d_920[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_711 (ReLU)               (None, 4, 4, 2048)   0           ['batch_normalization_711[0][0]']
                                                                                                  
 conv2d_921 (Conv2D)            (None, 4, 4, 2048)   4196352     ['re_lu_711[0][0]']              
                                                                                                  
 conv2d_918 (Conv2D)            (None, 4, 4, 2048)   4196352     ['re_lu_709[0][0]']              
                                                                                                  
 batch_normalization_712 (Batch  (None, 4, 4, 2048)  8192        ['conv2d_921[0][0]']             
 Normaliza

In [54]:
img_size = 299
batch_size = 64
init_filter = 0
epoch = 50
cardinality = 0
features = '-transfer'

module_url = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/2"

model = tf.keras.Sequential([
    # hub.KerasLayer載入的模型為Keras Layer
    hub.KerasLayer(module_url,
                   input_shape=(299, 299, 3),  # 模型輸入大小
                   output_shape=(2048,),   # 模型輸出大小
                   trainable=False),  # 將模型訓練權重設定為False(凍結)
    # 最後接上兩層全連接層
    layers.Dense(128, activation='relu'), 
    layers.Dense(11, activation='relu')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 2048)              21802784  
                                                                 
 dense_19 (Dense)            (None, 128)               262272    
                                                                 
 dense_20 (Dense)            (None, 11)                1419      
                                                                 
Total params: 22,066,475
Trainable params: 263,691
Non-trainable params: 21,802,784
_________________________________________________________________


In [25]:
%load_ext tensorboard
%tensorboard --logdir online_test-logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 97103), started 11:10:05 ago. (Use '!kill 97103' to kill it.)

In [55]:
run_name = 'img_size-'+str(img_size)+'-batch_size-'+str(batch_size)+'-init_filter-'+str(init_filter)+'-epoch-'+str(epoch)+features
model_dir = 'online_test/models' 
os.makedirs(model_dir,exist_ok=True) 
print(img_size)

# 儲存訓練記錄檔
logfiles = 'online_test-logs/{}'.format(run_name)
model_cbk = keras.callbacks.TensorBoard(log_dir=logfiles, 
                                        histogram_freq=1)
# 儲存最好的網路模型權重
modelfiles = model_dir + '/{}-best-model.online_test'.format(run_name)
model_mckp = keras.callbacks.ModelCheckpoint(modelfiles, 
                                             monitor='val_categorical_accuracy', 
                                             save_best_only=True, 
                                             mode='max')

# 設定訓練使用的優化器、損失函數和指標函數
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.CategoricalAccuracy()])

# 訓練網路模型
model.fit(train_set,
          epochs=epoch, 
          validation_data=valid_set,
          callbacks=[model_cbk, model_mckp])


299


"'model.fit(train_set,\n          epochs=epoch, \n          validation_data=valid_set,\n          callbacks=[model_cbk, model_mckp])"

In [51]:
#評估網路模型效果
model.load_weights(modelfiles)
loss, acc = model.evaluate(test_set)

2022-09-05 07:43:05.013356: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open online_test/models/img_size-128-batch_size-64-init_filter-64-epoch-50-transfer-best-model.online_test: FAILED_PRECONDITION: online_test/models/img_size-128-batch_size-64-init_filter-64-epoch-50-transfer-best-model.online_test; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


53/53 [==============================] - 56s 1s/step - loss: 2.7644 - categorical_accuracy: 0.5793
